# Functions María

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr, f_oneway, mannwhitneyu

### Review: describe_df()

In [ ]:
def describe_df(df):
    data_type = df.dtypes
    missings = df.isna().sum()/len(df) * 100
    unique_values = df.nunique()
    cardin = unique_values / len(df) * 100
    
    df_temp = pd.DataFrame({
        'DATA_TYPE': data_type,
        'MISSINGS (%)': missings,
        'UNIQUE_VALUES': unique_values,
        'CARDIN (%)': round(cardin, 2)
    })

    df_temp = df_temp.T
    return df_temp

### Review: tipifica_variables()

In [ ]:
def tipifica_variables(df, umbral_categoria, umbral_continua):
    types_dict = {}

    for col in df.columns:
        # Calculate cardinality for each column
        cardinality = df[col].nunique()
        # Calculate percentage cardinality for each column
        percentage_cardinality = cardinality / len(df) * 100

        # Classify each variable based on cardinality and percentage cardinality
        # Binary variables: 2 unique values
        if cardinality == 2:
            tipo = 'Binaraia'
        # Categorical variables: unique values less than or equal to 'umbral_categoria'
        elif cardinality <= umbral_categoria:
            tipo = 'Categorica'
        # Classify numeric variables: unique values greater than 'umbral_categoria'
        elif cardinality > umbral_categoria:
            # Numeric continuous: percentage cardinality greater than or equal to 'umbral_continua'
            if percentage_cardinality >= umbral_continua:
                tipo = 'Numerica Continua'
            # Numeric discrete: percentage cardinality below 'umbral_continua'
            else:
                tipo = 'Numerica Discreta'

        # Store proposed variable types in the dictionary 'types_dict' with column names as keys
        types_dict[col] = tipo

    # Create a DataFrame from the 'types_dict' dictionary with two columns: 'nombre_variable' and 'tipo_sugerido'
    df_temp = pd.DataFrame(types_dict.items(), columns=['nombre_variable', 'tipo_sugerido'])

    # Return the DataFrame
    return df_temp



### Review: get_features_num_regression()

In [ ]:
def get_features_num_regression(df, target_col, umbral_corr, pvalue=None, card=20):
    """
    Identifies and evaluates the correlation between numeric columns in a DataFrame and a specified target column.
    Stores and returns a list of columns that have an absolute Pearson correlation stat greater than a specified threshold ('umbral_corr').
    If a p-value is specified (pvalue) then this is used to check correlations for statistical signifcance and this is accounted for in column selection.

    Parameters:
    df (pd.DataFrame): DataFrame containing the data.
    target_col (str): Target column to correlate with numeric columns.
    umbral_corr (float): Correlation threshold (between 0 and 1) for the correlation test.
    pvalue (float, optional, Defaul=None): Signifance level (between 0 and 1) for the correlation test.
    card (int): Cardinality threshold checks for sufficient unique values in 'target_col'


    Returns:
    list ('features_num'): A list of columns that have correlated with target column above the specified threshold 'umbral_corr'
    """

    # First carry out checks to prevent errors
    #1. check df is a dataframe
    if not isinstance(df, pd.DataFrame):
        print("First arguement must be a Pandas DataFrame.")
        return None
    
    #2. check target_col is in df
    if target_col not in df.columns:
        print(f"The column '{target_col}' is not in the the specified DataFrame.")
        return None
    
    #3. check target_col is numeric and continuous (high cardinality)
    # https://pandas.pydata.org/docs/reference/api/pandas.api.types.is_numeric_dtype.html
    if not (pd.api.types.is_numeric_dtype(df[target_col]) and df[target_col].nunique() > card):
        print(f"The column '{target_col}' must be a continuous numeric variable with high cardinality. \nCheck the 'card' value.")
        return None
    
    # Check umbral_corr is float between 0 and 1 (and not (0 <= umbral_corr => 1)
    if not isinstance(umbral_corr, (int, float)) or not (0 <= umbral_corr <= 1):
        print("'umbral_corr' must be a number between 0 and 1.")
        return None
    
    # Check pvalue is float between 0 and 1 (and not (0 <= pvalue => 1)
    if pvalue is not None:
        if not isinstance(pvalue, (int, float)) or not (0 <= pvalue <= 1):
            print("'pvalue' must be 'None' or a number between 0 and 1.")
            return None
        
    #2. Initialize features list to store selected numeric features
    features_num = []

    #3. Loop over all numeric columns in the dataframe
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html
    for col in df.select_dtypes(include=[float, int]).columns:
        if col == target_col:
            continue

        # Calculate pearsonr corr stat and p_value
        corr, p_val = pearsonr(df[col], df[target_col])

        # Check corr stat is greater than 'umbral_corr'
        # Convert to absolute value to avoid problems with negative correlations
        if abs(corr) > umbral_corr:
            if pvalue is None:
                features_num.append(col)
            elif p_val <= 1 - pvalue:
                features_num.append(col)

    # Return the selected numeric columns list 'features_num'
    return features_num

### Review: plot_features_num_regression()

In [ ]:
def plot_features_num_regression(df, target_col="", card=20, columns=[], umbral_corr=0, pvalue=None):

    """
    Generates pair plots for numeric columns in a DataFrame based on their correlation with a specified target column.
    Pair plots are generated in maximum 5x5 grids.
    If specific numeric columns are not specified the function will filter the numeric columns in the DataFrame based on
    a specified correlation threshold ('umbral_corr') and optionally a p-value significance level.
    Checks the threshold conditions of specified columns and offers options to remove if columns are not valid or continue
    anyway.
    
    Parameters:
    df (pd.DataFrame): Dataframe containing data.
    target_col (str): The target column to correlate with other numeric columns. Must be numeric continuous variable with high cardinality.
    card (int): Cardinality threshold checks for sufficient unique values in 'target_col'
    umbral_corr (float): Correlation threshold (between 0 and 1) for correlation testing if numeric columns are not specified.
    pvalue (float, optional, Defaul=None): Signifance level (between 0 and 1) for the correlation testing if numeric columns are not specified.

    Returns:
    list ('columns'): List of columns used for generating the pair plots
    """

    # First carry out checks to prevent errors
    #1. Check df is a dataframe
    if not isinstance(df, pd.DataFrame):
        print('First arguement must be a Pandas DataFrame.')
        return None

    #2. Check target_col is in DataFrame, and is numeric and continuous (high cardinality)
    if target_col not in df.columns or not (pd.api.types.is_numeric_dtype(df[target_col]) and df[target_col].nunique() > card):
        print(f"The target column ('{target_col}') must be a numeric continuous variable with high cardinality.\nCheck 'card' value")
        return None
    
    #3. Check pvalue is float between 0 and 1 (and not (0 <= pvalue => 1)
    if pvalue is not None:
        if not isinstance(pvalue, (int, float)) or not (0 <= pvalue <= 1):
            print("'pvalue' must be 'None' or a number between 0 and 1.")
            return None

    # If no numeric columns are specified, get columns using function 'get_features_num_regression()' based on 'umbral_corr' and 'pvalue'
    if not columns:
        columns = get_features_num_regression(df=df, target_col=target_col, umbral_corr=umbral_corr, pvalue=pvalue)
    else:
        valid_cols = [] # Create empty list to store columns that meet threshold conditions
        for col in columns: # Loop through columns in columns list
            if col == target_col:
                continue # Skip the target column itself as already been checked for validity

            # Calculate pearsonr corr stat and p_value between column and target column
            corr, p_val = pearsonr(df[col], df[target_col])

            # Check corr stat and p-value meet specified thresholds
            if abs(corr) > umbral_corr:
                if pvalue is None or p_val <= pvalue:
                    valid_cols.append(col) # add column to valid_cols list if it meets both thresholds
                else:
                    # Warn that column does not meet the required p-value significance level
                    print(f"'{col}' did not meet the p-value signifcance level")
                    # Ask if you want to remove the column or continue anyway
                    question = input(f"Do you want to remove '{col}' from the columns list or continue anyway? Type 'remove' or 'continue'").strip().lower()

                    if question == 'continue': 
                        valid_cols.append(col) # adds column to valid_cols list if user types continue
                    else:
                        print(f"'{col}' was removed from columns list")
                        continue
            
            else:
                # Warn that column does not meet the required correlation threshold
                print(f"'{col}' did not meet the correlation threshold of {umbral_corr}.")
                # Ask if you want to remove the column or continue anyway
                question = input(f"Do you want to remove '{col}' from the columns list or continue anyway? Type 'remove' or 'continue'").strip().lower()
                if question == 'continue':
                    valid_cols.append(col) # adds column to valid_cols list if user types continue
                else:
                    print(f"'{col}' was removed from columns list")
                    continue
        
        if valid_cols: # Check there are still valid columns left in valid_cols
            columns = valid_cols # Sets columns to valid_columns after checks and warnings
        else:
            columns = get_features_num_regression(df=df, target_col=target_col, umbral_corr=umbral_corr, pvalue=pvalue)

    columns = [col for col in columns if col != target_col] # Make sure target is not in columns list to plot
    print(f"columns selected for pair plot analysis were: {columns}")
    
    # Generate pair plots in max 5x5 grids
    for i in range(0, len(columns), 4):
        sns.pairplot(df, vars=[target_col] + columns[i:i + 4])
        plt.show()

    # Return the selected numeric columns list 'columns'
    return columns

### Review: get_features_cat_regression()

In [ ]:
# from scipy.stats import f_oneway, mannwhitneyu
# possibile addition? --> see comment marked ???
def get_features_cat_regression(df, target_col, pvalue=0.05, card=20):
    """
    Identifies and evaluates the significance of relationship between categorical columns and a specified numeric target column in a DataFrame.
    Uses ANOVA for multi-cats or Mann_whitney U for binary-cats
    Stores and returns a list of columns that have show a significant relationship with target column based on spcifed (optionally) pvalue.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing the data
    target_col (str): Numeric target column for testing relationship with categorical columns
    pvalue (float, optional, Default=0.05): Significance level (between 0 and 1) for statistical test evaluation.
    card (int): Cardinality threshold (based on unique values) to determine if a column should be considered categorical.

    Returns:
    list ('categorical_features'): A list of categorical columns that have a significant relationship with target column based on pvalue arguement.
    """
    # Carry out input data checks
    #1. Check df is a dataframe
    if not isinstance(df, pd.DataFrame):
        print('First arguement must be a Pandas DataFrame.')
        return None
    
    #2. Check target column in DataFrame
    if not target_col in df.columns:
        print(f"The target column ('{target_col}') must be in the DataFrame.")
        return None
    
    # Check target column is numeric and has sufficiently high cardinality
    if not (pd.api.types.is_numeric_dtype(df[target_col]) and df[target_col].nunique() > card):
        print(f"The target column ('{target_col}') must be a numeric continuous variable with high cardinality.\nCheck 'card' value")

    # Check pvalue is float between 0 and 1 (and not (0 <= pvalue => 1)
    if pvalue is not None:
        if not isinstance(pvalue, (int, float)) or not (0 <= pvalue <= 1):
            print("'pvalue' must be a 'None' or a number between 0 and 1.")
            return None
    
    # Create empty list to store columns considered to have statistically significant relationship with target column
    categorical_features = []

    # Loop through each column in the DataFrame
    for col in df.columns:
        if col == target_col: # Skip target column itself
            continue
        
        # Check the cardinality of column to decide if categorical or not
        if len(df[col].unique()) <= card: #??? Could we add if 'df[col].dtype == 'object' to this if?
            # If categorical and binary perform Mann-Whitney U test
            if df[col].nunique() == 2:
                groupA = df[df[col] == df[col].unique()[0]][target_col]
                groupB = df[df[col] == df[col].unique()[1]][target_col]

                p_val = mannwhitneyu(groupA, groupB).pvalue
            
            else:
                # If categorical with more than 2 groups, perform ANOVA test
                groups = df[col].unique()
                target_by_groups = [df[df[col] == group][target_col] for group in groups]

                p_val = f_oneway(*target_by_groups).pvalue

            # Check p-val against pvalue arguement to see if significance threshold is met
            if p_val <= pvalue:
                categorical_features.append(col) # Add to categorical_features list if deemed significant

    # Return list of categorical features
    return categorical_features

### Review: plot_features_cat_regression()

Esta función recibe un dataframe, una argumento "target_col" con valor por defecto "", una lista de strings ("columns") cuyo valor por defecto es la lista vacía, un argumento ("pvalue") con valor 0.05 por defecto y un argumento "with_individual_plot" a False.

Si la lista no está vacía, la función pintará los histogramas agrupados de la variable "target_col" para cada uno de los valores de las variables categóricas incluidas en columns que cumplan que su test de relación con "target_col" es significatio para el nivel 1-pvalue de significación estadística. La función devolverá los valores de "columns" que cumplan con las condiciones anteriores. 

Si la lista está vacía, entonces la función igualará "columns" a las variables numéricas del dataframe y se comportará como se describe en el párrafo anterior.

De igual manera que en la función descrita anteriormente deberá hacer un check de los valores de entrada y comportarse como se describe en el último párrafo de la función `get_features_cat_regression`.